In [7]:
%%bash
# SR from https://dodcio.defense.gov/Portals/0/Documents/CMMC/AssessmentGuideL2v2.pdf
jq -s '[.[]] | flatten' r2/*.json > r2.json

In [8]:
import requests
import os
import json
import re 

# URL to fetch the JSON data
url = "https://csrc.nist.gov/extensions/nudp/services/json/nudp/framework/version/sp_800_171_2_0_0/export/json?element=all"

# File path to store the JSON data
framework_file_path = "../client/public/data/sp_800_171_2_0_0/framework.json"

# Fetch the JSON data
response = requests.get(url)
framework_data = response.json()

# Regex match form 3.14.1 and 03.14
pattern = re.compile(r'\d{1}\.\d{1,2}(?:\.\d{1,2})?')

# Convert v2 id to v3
for element in framework_data["response"]["elements"]["elements"]:
    identifier = pattern.search(element['element_identifier'])
    if identifier:
        v3_identifier = ".".join(map(lambda c: c.rjust(2, '0'), identifier.group(0).split(".")))
        element['element_identifier'] = element['element_identifier'].replace(identifier.group(0), v3_identifier)

with open('r2.json', 'r') as r2:
    framework_data['response']['elements']['elements'] = framework_data['response']['elements']['elements'] + json.load(r2)


# Ensure the directory exists
os.makedirs(os.path.dirname(framework_file_path), exist_ok=True)

# Write the JSON data to the file
with open(framework_file_path, 'w') as file:
    json.dump(framework_data, file, indent=4)

print(f"JSON data has been stored in {framework_file_path}")

JSON data has been stored in ../client/public/data/sp_800_171_2_0_0/framework.json
